### Main method: running TA Chatbot

Implemented:
* Contriever (IR)
* OPT (generation)
* MS_Marco (ranking)
* Separately, DocQuery retrieves "quotes" the textbook. It gives short, direct answers.

Todo: 
* Implement Flan-T5 (instruction-finetuned) instead of, or _maybe_ in addition to, OPT. 
* UI/UX.
  * Enable user-provided contexts to return direct answers about a piece of text they are studying.
* Fine-tuned models. 

Dream ideas:
* Implement roberta QA Pipeline (from Jerome's DocQuery notebook)
* Implement huggingface SentenceTransformers 
* Implement context search from GPT embeddings 'QA' specialized models.

TO RUN: you must clone all our repos into the same top directory. 

Required directory structure:
```text
-- ta_chatbot
---- main_fun
---- data-generator
---- info-retrieval
```

In [3]:
import torch
import main
USER_QUESTION = "user_question = 'What is the Moore machine?'"

In [ ]:
# retriever: Xiang: RAG (DPR), Kastan: Contriver
# generator: Xiang: T5/OPT (Fine-tuned), Kastan: OPT (Pre-trained)
# question re-write: Xiang: T5 (Fine-tuned), Kastan: NONE
# re-ranker: Xiang: NONE, Kastan: MS-Marco (Fine-tuned)

In [6]:
NUM_ANSWERS_GENERATED = 5

# init our Main() class -- all our models are class properties
ta = main.TA_Pipeline()
# contriever: find relevant passages (retriever: document + question = 5 passages)
top_context_list = ta.contriever(user_question=USER_QUESTION, num_answers_generated=NUM_ANSWERS_GENERATED)
# opt-1.3B: 5 passage + 1 question = 5 answer
generated_answers_list = ta.OPT(top_context_list)
# rank 5 answers
scores = ta.re_ranking_ms_marco(generated_answers_list)
# print best answer
index_of_best_answer = torch.argmax(scores) # get best answer
print("\n-------------------------------------------------------------\n")
print("Question: \n", USER_QUESTION)
print("Best answer: \n", generated_answers_list[index_of_best_answer])

User question:  user_question = 'When did Turing invent the Turing machine?'
Generated Answers:
  > On June 18, 1953 in the United Kingdom's West Cotswold village of Ovinghoe he made a breakthrough that helped pave the way for computers to be used by scientists around the world.<br />(http://en.wikipedia.org/wiki/Timothy%20Haldane#Philosophy)<br /><p>Since our first post-turing machine was an IBM 703 (a 704 with a special chip for arithmetic operations only), we have no idea what his first invention really was. That is because there are not many examples of it existing. Even fewer exist today than when Turing wrote that famous statement.</p>
<h2 style="text-align: justify;">More Information</h2>
<ul style="line-height: 115%;"><li align="right">Turing's other inventions</li></ul>
<ol style="line-height: 110%; width: 100%; height: 80px;" id="startAlign" class="MsoListParagraphCxSpMiddleColumn"><th style="text-decoration: underline;">Start Column - What are they?</th>
</ol><pre class="Mso

In [5]:
import torch
import main

# init our Main() class -- all our models are class properties
USER_QUESTION = "user_question = 'When did Turing invent the Turing machine?'"
ta = main.TA_Pipeline()
NUM_ANSWERS_GENERATED = 5

answer = ta.doc_query(user_question=USER_QUESTION, num_answers_generated=NUM_ANSWERS_GENERATED)
print("The user question is: ", USER_QUESTION)
print("DocQuery's concise, direct quote, answer: \n", answer)

The user question is:  user_question = 'When did Turing invent the Turing machine?'
DocQuery's concise, direct quote, answer: 
 3.6.2


#### UX via Gradio

In [13]:
import gradio as gr
import numpy as np

## DOCS: https://gradio.app/getting_started/
## See figma for our ideal design.

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="text",
)

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image")
demo.launch(share=True)

# demo.launch()
# ssh -L 7860:localhost:7860 kastan@kastan

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://02ba4546d19a31bc.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fb1042545e0>,
 'http://127.0.0.1:7861/',
 'https://02ba4546d19a31bc.gradio.app')

Traceback (most recent call last):
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/gradio/routes.py", line 289, in run_predict
    output = await app.blocks.process_api(
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/gradio/blocks.py", line 982, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/gradio/blocks.py", line 824, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/haob2/anaconda3/envs/taqa/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = conte